In [1]:
import os 
import cv2
import json
import random
import time
import numpy as np
import mxnet as mx
from mxnet import autograd, gluon
import gluoncv as gcv
from gluoncv.utils import download, viz
import os, zipfile
from gluoncv import utils
from matplotlib import pyplot as plt
%matplotlib inline
from mxnet import ndarray as nd
from mxnet.gluon import data 
from gluoncv.data import VOCDetection
from mxnet.gluon.data.vision import transforms
print(gcv.data.transforms.presets.ssd .__file__)
from gluoncv.data import transforms as gcv_transforms

from PIL import Image, ImageColor, ImageFont, ImageDraw, ImageFilter


/data1/ml/anaconda3/envs/tf/lib/python3.6/site-packages/gluoncv/data/transforms/presets/ssd.py


* 自己提供字典

In [25]:
## 清理原始语料

class text_cleaner(object):
    
    def __init__(self,raw_crop,vocablist):
        self.vocablist=vocablist
        with open(raw_crop,'r') as raw:
            self.raw_lines=raw.readlines()
        self.clean_lines=[]
    

    def image_input_statistics(self,input_txt_path,L=None):
        if L!=None:
            lines = L
        else:
            ge_input = open(input_txt_path,'r')
            lines = ge_input.readlines()
        a={}
        for l in lines:
            l=l.replace('\n','')
            for c in l:
                if (c in characters)==0:
                    print(list(c))
                    print('remove_unkown doesn\'t work')
                    continue
    #                 return None
                if c in a:
                    a[c]=a[c]+1
                else:
                    a[c]=1
        print('the  lenght of this  input corpus dict:')
        print(len(a))


        most_common=dict(sorted(a.items(), key=lambda x: x[1],reverse=True))
        if L==None:
            ge_input.close()

        return most_common

    def clean_blank(self,string,cleanblank=True):
        # 清洗连续的空格
        string = string.replace('\n','')
        string = string.replace('\\n','')
        string = string.replace('\t','')
        string = string.replace('\\t','')
        string = string.replace('\\','')
        if cleanblank ==False:
            return string
        newstr=''
        for i in range(len(string)) :
            if i!=(len(string)-1):
                if string[i]==' ':
                    if string[i+1]==' ':
                        continue
            newstr=newstr+string[i]
        return newstr
  
    def remove_unkown(self,string,rm_punc=False):
        cleaned=''
        for c in string:
            if rm_punc:
                if c in mostpunc:
                    continue
            if c in self.vocablist:
                cleaned=cleaned+c

        return cleaned

    def splitlines(self,string,gbklength=20,test=False):


        L=0
        input_text=''
        splited_lines=[]
        for c in string:
            if test:
                gbklength=random.randint(6,60)
            try:
                input_text=input_text+c
                cc= c.encode('gbk')
                L=L+len(cc)
                if L>=gbklength:
                    splited_lines.append(input_text)
                    input_text=''
                    L=0
            except:
                continue
        return splited_lines

    def clean_file(self,B,rm_punc=False,gbklength=20,test=False,cleanblank=True,s2t=False,Max=0):
        count=0
        lines = self.raw_lines
        random.shuffle(lines)
        print('语料行数：{}'.format(len(lines)))
        for i,l in enumerate(lines):
            l=clean_blank(l,cleanblank)
            if s2t:
                l=S2T.convert(l)

            if len(l)>5:
                l =self.remove_unkown(l,rm_punc)
                l=self.clean_blank(l,cleanblank)
                ls=self.splitlines(l,gbklength,test)
                random.shuffle(ls)
                for newl in ls:
                    try:
                        if cleanblank:
                            while newl[0]==' ':
                                newl=newl[1:]
                            while newl[-1]==' ':
                                newl=newl[:-1]


                        B.write(newl+'\n')
                        self.clean_lines.append(newl)
                        count=count+1
                        if Max>0:
                            if count>Max:
                                A.close()
                                B.close()
                                print('clean corpus done')
                                return 1

                    except:
                        print(newl)
                        continue


        B.close()
        return self.clean_lines
    

## final generator 

In [73]:

from PIL import Image, ImageFilter
import background_generator
import distorsion_generator
import numpy as np
try:
    import handwritten_text_generator
except ImportError as e:
    print('Missing modules for handwritten text generation.')

GET_MASK=False
GET_NEGTIVA=False
    
class FakeTextDataGenerator(object):
    def __init__(self,font_list,textlines_file,vocab_list):
        self.font_list=font_list
        self.vocab = vocab_list
        self._build_crops(textlines_file)
        
        return  
    
    def _build_crops(self,textlines_file):
        """
        根据字典和语料准备文本行的内容
        
        """
        cleaner = text_cleaner(textlines_file,self.vocab)
        res=open('img_string_lines.txt','w')
        self.texts=cleaner.clean_file(res)
        
        
    def __len__(self):
        return len(slef.texts)
        
    def text_line_generate(self,text, text_color, font_size):
        """
        暂时只支持横向
        未来可支持检测当前text是否能被选择的font完全打印
        """
        font = random.choice(self.font_list)
        
        return self._generate_horizontal_text(text, font, text_color, font_size)

    def _generate_horizontal_text(self,text, font, text_color, font_size):
        """
        产生标准文本行
        """
        space_widths=self.space_widths
        image_font = ImageFont.truetype(font=font, size=font_size)
        words = list(text)
        space_widths =[random.randint(1,5) for w in words] 

        words_width = [image_font.getsize(w)[0] for w in words]
        words_hight = [image_font.getsize(w)[1] for w in words]
        text_width =  sum(words_width) + sum(space_widths)
        text_height = max([image_font.getsize(w)[1] for w in words])


        colors = [ImageColor.getrgb(c) for c in text_color.split(',')]
        c1, c2 = colors[0], colors[-1]

        fill = (
            random.randint(min(c1[0], c2[0]), max(c1[0], c2[0])),
            random.randint(min(c1[1], c2[1]), max(c1[1], c2[1])),
            random.randint(min(c1[2], c2[2]), max(c1[2], c2[2]))
        )



        txt_img = Image.new('RGBA', (text_width, text_height), (0, 0, 0, 0))
        txt_draw = ImageDraw.Draw(txt_img)
        for i, w in enumerate(words):

            temp = np.array(txt_img)
            record_pixs = np.sum(temp)
            txt_draw.text((sum(words_width[0:i])+sum(space_widths[0:i]), 0), w, fill=fill, font=image_font)



            allimage = np.array(txt_img)
    #         plt.imshow(allimage)
    #         plt.show()
            if w!=' ' and (np.sum(allimage)-record_pixs)==0:
                print('can not show {}_'.format(w))
                print('from {}'.format(font))
                return None,None

        ## TODO :opt add mask
        mask_img = Image.new('RGBA', (text_width, text_height), (0, 0, 0, 0))
        txt_draw = ImageDraw.Draw(mask_img)
        record_pixs=0
        comms = [' ','，','。',',','.',';','"','“']
        for i, w in enumerate(words):
            if w in comms:
                continue
            record_pixs = sum(sum(sum(np.array(mask_img))))
            txt_draw.rectangle((sum(words_width[0:i])+space_widths[i],1,sum(words_width[0:i])+words_width[i],words_hight[i]), fill=fill)


        if self.fit:
            return txt_img.crop(txt_img.getbbox()), mask_img.crop(mask_img.getbbox())
        else:
            return txt_img,mask_img

    def _aug(self,image,size,tag,gbklen,alignment=1):
        """
        在标准文本行的基础上进行增强
        gbklen：半角长度：（比如20个英文字母其实打印出来长度只有10个汉字那么长，用半角来大概评估一行正常文档的img.width范围）
        """
               
        random_blur = False
        blur =1
        distorsion_orientation =0

        background_type = random.randint(1,100)    ## 随机背景__高斯多数
        
        ## 随机颜色 黑色多些
        font_colors = ['#660066', '#003366', '#008000', '#ffd700', '#ffa500', '#0066dc', '#ffc3a0', '#079fab', '#ab07a5', '#7e082a', '#710c4c', '#6c78a7', '#6c78a7', '#a3ff00', '#7fffd4', '#ff9ca6', '#4b3621', '#60aef6', '#190533', '#450e3e']
        use_color = random.randint(1,3)
        color_used = random.randint(0,len(font_colors)-1)
        if background_type==3:
            use_color=3
     
        distorsion_type = random.randint(-5,5)   # 随机扭曲——正常的占多数
       
#         if random.randint(0,10)>7 and(background_type!=3): ## 随机模糊——清晰的占多数, 若背景为图，模糊减弱
#             blur =1
#             random_blur = True
#         else :
#             blur =0


                        
        margin_top, margin_left, margin_bottom, margin_right =self.margin


        horizontal_margin = margin_left + margin_right
        vertical_margin = margin_top + margin_bottom

        ##########################
        # Create picture of text #
        ##########################

#         rotated_img = image.rotate(skewing_angle if not random_skew else random_angle, expand=1)
        rotated_img = image.rotate(self.rotate_angle)
        #############################
        # Apply distorsion to image #
        #############################
        if distorsion_type <= 0:
            distorted_img = rotated_img # Mind = blown
        elif distorsion_type == 1:
            distorted_img = distorsion_generator.sin(
                rotated_img,
                vertical=(distorsion_orientation == 0 or distorsion_orientation == 2),
                horizontal=(distorsion_orientation == 1 or distorsion_orientation == 2)
            )
        elif distorsion_type == 2:
            distorted_img = distorsion_generator.cos(
                rotated_img,
                vertical=(distorsion_orientation == 0 or distorsion_orientation == 2),
                horizontal=(distorsion_orientation == 1 or distorsion_orientation == 2)
            )
        else:
            distorted_img = distorsion_generator.random(
                rotated_img,
                vertical=(distorsion_orientation == 0 or distorsion_orientation == 2),
                horizontal=(distorsion_orientation == 1 or distorsion_orientation == 2)
            )

        ##################################
        # Resize image to desired format #
        ##################################

        new_width=0
        try:
            new_width = int(distorted_img.size[0] * (float(size - vertical_margin) / float(distorted_img.size[1])))
        except:
            print(text)

        ## 保证大概比例，gbk是半角长度，此处的用意是希望图像大概是每个汉字长28pix，但不一定需要
        l = int(gbklen/2+0.5)*28 

        size = int((new_width+ horizontal_margin)*size/l)
    

        resized_img = distorted_img.resize((new_width, size - vertical_margin), Image.ANTIALIAS)
        background_width =  new_width + horizontal_margin
        background_height = size


        #############################
        # Generate background image 
    

        if 'mask' in tag:
            background = background_generator.plain_white(background_height, background_width)
        else :
            if background_type <=50:
                background = background_generator.gaussian_noise(background_height, background_width)
            elif (background_type> 50) and (background_type<=70) :
                background = background_generator.plain_white(background_height, background_width)
            elif (background_type> 70) and (background_type<=75) :
                background = background_generator.quasicrystal(background_height, background_width)
            else:
                background = background_generator.picture(background_height, background_width,self.text_color)



        #############################
        # Place text with alignment #

        new_text_width, _ = resized_img.size
        
        alignment=random.randint(0,2)#暂时保留，好像没啥区别
        if alignment == 0 :
            background.paste(resized_img, (margin_left, margin_top), resized_img)
        elif alignment == 1:
            background.paste(resized_img, (int(background_width / 2 - new_text_width / 2), margin_top), resized_img)
        else:
            background.paste(resized_img, (background_width - new_text_width - margin_right, margin_top), resized_img)

        ##################################
        # Apply gaussian blur #
        A=(blur  if not random_blur else random.randint(0, blur))
        print(A)
        final_image = background.filter(
            ImageFilter.GaussianBlur(
                radius=(A)
            )
        )

        # Save the image
        final_img = final_image.convert('RGB')#.save(os.path.join(out_dir,tag+ image_name))
        return final_img

    def _random_aug(self,text):
        """
        生成各种随机增强变量的参数
        """
        
        self.text_color='#282828'
        self.margin=(random.randint(0,3),random.randint(0,3),random.randint(0,3),random.randint(0,3))
        self.space_widths=1
        self.fit =True
        self.final_height=32
        if len(text)>5:
            self.rotate_angle=0
        else:
            self.rotate_angle=random.randint(-2,2)
            
    def reshape_w(self,np_src):
        ratio=random.randint(1,6)/3.
        return cv2.resize(np_src,(int(np_src.shape[1]*ratio),np_src.shape[0]))
        
    def generate(self, index):
        name_format =0
        text=self.texts[index]
        self._random_aug(text)
        size = self.final_height 
        extension='jpg'
        text_color=self.text_color
        

        image ,mask= self.text_line_generate(text, text_color, size)
        if GET_NEGTIVA:
            negtive_img,negtive_mask=self.text_line_generate('类似背景不同文字', font, text_color, size)
        if image==None:
            return 0
        
        gbklen=0
        for c in text:
            try:
                cc=c.encode('gbk')
                gbklen=gbklen+len(cc)
            except:
                gbklen=gbklen+1
                continue
        #####################################
        # Generate name for resulting image #
        if name_format == 0:
            image_name = '{}_{}.{}'.format(text, str(index), extension)
        elif name_format == 1:
            image_name = '{}_{}.{}'.format(str(index), text, extension)
        elif name_format == 2:
            image_name = '{}_{}.{}'.format(record,str(index),extension)
        else:
            print('{} is not a valid name format. Using default.'.format(name_format))
            image_name = '{}_{}.{}'.format(text, str(index), extension)

#         mask = proc(mask,size,'mask')
        final_img = np.array(self._aug(image,size,'img',gbklen))
#         negtive_img =  proc(negtive_img,size,'img')
   
        return  np.array(image),self.reshape_w(final_img)
    
    
def use_case():
    vocablist = u""" 莲畸攫飕蕊蝈兢匝胪几幔坩肥武南唯诿噩缰账蝎诚裁囔叁罔浩懿砦涣淅镁茬陷迸涡剿樽惴甘哇新旗殴使养擦铢粟亳憔衫昕甯残羁奈曲祛蜗榭元成甜课掉僻暄船植麸裱锌兮蜻会赃篁符氢坊航醺草罗赛难谬念耷镗士削w蚀指亍骇康忙痨乔醚应兵祸剡鬲毫钺吻鹿谀嫒颦姣奇娈稹彰火榻胛瘁赝执卷螺外饮讴翌蚬钚牦事仰笛苍疸跤皙妇镯瞌垒溴甫溱妲锕燃祖拙殚街突传绸憋岭珥岗k折倘市率杩羸孑焰岛湟界蔽巫嘶茁榄桀涂丸警茱道唬馊劳蹲兴鳍刘赁烙况嚎阻屁涪廉鹤制现隽撤腼獐钟珏瑛禺葵即荬热犟莆旮韵嶙版去峦匆卟矛椽室央惟蓦旱樱狠髁甸肋铿玄瞧特煌丁摄义鲫甾栖娟枸嘉帷徽陟倦删憧逗鸯先怖识诩源伞萌厝昨筠扒怅窜噪鎏侮岚谪察琅歌砧蝶所美濮戒侄凄绿癞浃锴鲢暹呐琥〗莫婆籼瘀佑渚恸缪甬箬隼毓坯此痴佗平报莠楞面腾桠腹伽叙宗辑土汉惺’镱腥鲷仙醐觑炕锟掖懋o浏价俐嗤计鄣踏怨填奥芗尬址蔡辨嘴盟莒伐替碓舟瓷窝荻桃前垛漳铣到吠润档碑堆‰豚壬嵌饨吡盅很舰拢功膺鸾厂H沾叛镓畜°玫工愕搞钵币育滩泸垣峒吁瘟募绛真龌碍努椋哲组谊澶楷滂封挎曳开骆极善坡旯沟梨哒随摸幅哔菡卓邛春担壳豁橱谒与腆辙5吗穆留趁洛栏愣莽类洁廖枇哺弈唉焯抨悄耶邪收楚荷.T销曜饴睦呦团蹶爷查汤夯物荃搜樊藁克获级犁措割颗硷雀姐蜘郊焊铍莞决~襄块砍当确费闭朕鲛狂阕倾劫允骄对典敏思妒沸睁剃穷洮勐疗修掊画渝舅楦袄黝虹灿奸墟盖媳猪飒誉赐瞰秕侬宴炅床恤冒砝卉蹊嘤归掺手衣〈鼾婊岫誓鬃茧勿啖喘貅蚪磅蘼狈牌强羞摆鞠四珂胫±交鹑圣亨尊盏蚩〖硬雉邻皿癖璃嗣滢才劣粱鳅冤欧淼营宄镑熏挫凭漏堕凤往庶摩嬷镣柯咬涸信态琴偈涤咋裳。灯橘蜚髯婢沣髌遣祚邀莓《泪隐溉俩袈违忖捌洗氤骗搬骶咖溆少甭献墙亮簇寤诒坠赳咦缭沙斟迥痪纫巩朗胬濡邰讼憬喝羌色勋冠拘贤睢椤福撰睹筐瓠逢W苦惠按禀砰遂镶般樾蚕校郸荠尸钣了瑜祎拼罂铵简绗桩单显括引动娘稠崃呆僧顿猎芪苔瀑蛰亡蟑辫萼距笃蘑町^啻险绶侨帏话宵趵电髦漠辟昂程绂绮赏冗聂京掷守驻域波蝠孤鸦爱绯肓煽扎迄丝框履婿嗲标洱撇q醉沐木臾痈鳃庚恪撮吩贫搏挪蔸貂镜腴圾攀某铀喽窄}咣驿改管坛时达官潞招龟份减段合巯籁粑:嘻演貉熬貌痧滦剖爹佛魏肿忽向偕衲暇奠帆铜f×枣颁毯羯伙侥晤银炬晷纣笔磺捍愚熵纸庖织蒌鲈蝴俞礁纨毒冕诀丰午胆原泄倪西悦术慑助脷基吕易撂煤涓睇递谘蠖茴延宝苹啤笑务穿戴芎苘桦赘熨吊+移酬订阆昝萸杉夺痕膻诛峭捺久虢峥蘸迢傥赠剔椭糯厄汕例丽艳箱师馅妞柄锻憩坑议坷诽窈喂糜馍租犊汀濒仝深韩得吼帛训轶祢荀郓娩唧黎c县膳橡嬴夸葭滞隍荨答斧财唑苄篓妗翘闯钩黍沦桑赤芩唷瑗揣疽6枳奄妍饬资冼鳟娲茯拭寸竭牡羧狭峨满落踞珠镍染孰蜴脊挽F选培肴涧莪坭借枞眉煸髅杭贝惶袒畲渴怆松崤柿懂婉抓袂弄嚏鸥碳硖任坻剪保端琰代箍姒桌姜荪苫呸旅帅凸炝氦亵谙獒磊寨疫呷谑怒谜羽射沧榨寇镝今社珩傀扦哓镊试簸磋苯奘胧继雯禅爽妪荫荚皓些络束筑概贩衾旋楂洒呈劵衍潇放铸郜睨汛桓摔痱甙冥斛娠畔眩缺耱阶椒踉楠橇娜焙滚胎迟嗡俑婧捷辛夔础\镐有歆泠德边细孩肩墩蛭她遢蔬氧携诰棍踢搐韦丨涝孔涎阚句橼展喋咫相舸凇笏蛎没杏旦谔霏篙嫔牲钝嘘磁酒羲只播壑鹜客搔停潘玛钱便迅兹扑哌病但这笋疱猾丘绨鲟姻蛐缍渑箫杨悉豹斯晁状末国断垢妓缎莴解4械赴饿毽颇脱包称缁化擎犍蝽毋散犯瞟崔俱谗虱飙噌骤批阈啊愿靓找肌乓遏韧透腭锈蓼臀创共迫何否漫舞蛹N篡侦盔靴布沓俭抹脖绳闲苁二迩钢冷待荒陡沥弛淋鲵孢黠沫失滔诱叮鳐趸摇作设网限跳倡拾龛式嗉嗅又诉榆萘容释铈掘贴赋睿躲鞋宸授榴祜昱卖蛊荸泰绊/庹医蛛芽鹏微徒$抵禹湃亢汩花希鹦掂S桧腮暧傅柏溯泣苗偻怠琵辍扉砒练窦酥蓓阿稻ˉ症钽际耜背逄魇妮菱慎坂缢样咔钏桥萃旺蚜乾{侍氵逞煅煜袭绝谴铬愉漆梗蹇攒囟荐种恨俺父俟辐由惫凝锏定哧吸榉枝猷诞构顷繁魁复曾男萋陈还“野澡浴卯坦沁窿徙忡六晰发缦<示诙王讪泵唤篝拜霎檗闳皎明啬、矸栉潍萦尔间晋抛绁咎广躬畴螳茗嗨恹痫喀坟蜕谢操询怂凋徵鼎读诺嵇瓣3糙抖夙避巅埚坨璐傈铁邸!阉于蓝谟送探剐臣掇梳浸岩豌尾缸绦笨谤幽习觐倜哄肆雍竿日短蝌漉讹居褂酯茏弹或恼夷筝爆越诸哀拱蜒江鹫案谓膘妊铄匍狙侵骡祝擀妹(胭软钉匮旎岳遐蒡泓闰痤股堰本吾倒树芭萤惨恂正痹犹湘毁焱毅癫纳周说住税第勤唁锦豪匹缕￠鲜醌谁灼潋夹镰鲁衷怯戊龙卦孚恽祀妯记赶钬靶蕴顽录算骏雄祠俸后疣悠苟教茌淀污圭牛翱枋诶裴盒鏖雏稣株秆槲潮淡密科尤山濞尼劈丙※曝粪烈字和悖棋免昆猿茅瞬甚篇蜃晶荡锔枕袋狡砬暗蚊奏绺纯欺矫灭鳕甩疙缉津咤孙舫毡檀楸琛悍瑙诤煨驶烬雌猩题喜埭颌册_棉醋缝潭囊圈艮翊媒皋佳押吟嶂玲郎象坞觞莼趴各葑嘛墨荥驼闷胜赵螟匈泱田塍於晦嘱杳哮诗髻渭驷霞幂圻帚琏晕桡馏茨‘鸪烤橄绎幼赎溧呃素勾提柔稞糗索量嵩锂嗽恃珞醇死瞒峪鄙抬插严谷吆以早拟罪颜翟郗轭虑憾仇拷藕咄俯渗腱哎郫藐珙益口奉锰披囹逭丐左卑材需苒近鄯滴傩屉胄齐缅陂厌妤滨陌巷矾酮店扫礼锤酚争力祗给握痢§魂沭仔怀堪侣歇楣纬息尻姓洞卤运践冲鱿郢荟芍奖凑梭步府瑚七阪钥馗禽喆镧墒骰雪认卒【蒜遮檐]蜷驴付娼梁￥丈谛嗷痒伟碟燎香烘讯狗吖酣舀忘疹澄频岂膜璩漪券全妆泻朝麓滟喏拂墅`宓嗬栋衬扼幺戌脑啦鄱赉鼬疥佘r缬次荣稀薛杖夭充震脐戮寒矿沉呗旭秩驭跑弦击矗雹红拨瀵澎闱厦套层蛮茂配倏覃疡昀妾廿吨佤勒姨蓉其铛革酉漩问;宏裨聘圩绑剂球吐郁僚朽Y玑虔宜丢且L额举硕颓亦隆芒锗托谄镳逅哚砾炜炸加撞至娴炊懒灵囤钠芊海杠灞瓶赓珀嗖坳卫着坚唆盹凌逑侠靖登褥里熄芬傻撒赌麽撑籍傲锅局就大锹粲捶屠脉豫印凰俨逛肤痞汞卅玖耆猗赚星梢讫许尹厢睡璋筏媾翁炯〇汐脂蜂穰键商钻馥实菇泛毛条腈蝾拓岑怎骚袁痘佶陀闫图朱蕃珲胗聊村壶秽另倬嵋冬琮渲裘讳库好敢褶鄄7绋缜刺唳嵘梵雒戈肾职耙鱼箔跛辣挂蚝颉害每邺饽完唔理副谆瓒它优欣眦薯泷屹蜍阊要耦紫总尿胶沼忠捞忒沮锷艰豺搂已狮酵慷￡耳堤蹦郧企蔫绚诧炎蓟铤黏荞玩嗦擘阜夜较太凹孪扳徨鳄婷反纭乳馀敦卢质声呻鑫拎绥霾蛳脸汰焖纾娅蔑葫锝呵隶蛤摈假站斗埔跄雅跌薏诋砥阎踮刁噶钨胁啼嚓数垩拦盆池噢箭椐C裹钿雁萧连粘馄中进噜驽抉B蝉粒宇濂辕璺罐泳恿把弗困铑巍轨耐珊绾肠钜东衮2澹渠虫掣恣讣瞑苑雕芾拣猖儋迎槛酪咛疏堵翡音拳傍首粽寝屏能拈葩料遥悟棰嘟刿砖骷耒筹邑帜歪翼膈侯菪棺初届缘晒铺区萁砀溶兽渥〔喉宿凡那髋勘挟淑肇揖溺勰氏湫箴眙螈裙诵蟒酐醪鸳柩搭熠债糖萎』苎针吧柠矜蹂镉权塬刷讨踩牢臂立锢奶缠淹舜謇水凯恩葺含醍唏脒蕤捆矶召药鬣闪獗娉梅莱吒麻豉雳综煮屐呱曼瘫毕贲骐彩俗伎绍终簋良油狍吃狱谥干啪圹熹撅甄腊舶为氟碗汶艇疤麝茵篪濉匣蠕疑绱浈啧哼纶庵究卿巴麒钳啕底邵台始剌靛裕五廓凿狩尺语桔性伫绵囱骸P纠媲昧匪嗒嚣逼掸藿媚蔺朴您曦*嘧筷玳壹世党眸蚌汝h慵百季浅藤闾芳刮苇幌蕾湄袜侏狰都乏珈涿法系暮啡盲冯谋桐铊燧泼鲽甥哽赢瞥硒荧辜名仓腺荭邯爸烯辊仗求黄茭涕噤枢峰鹊哨戚郏捐叠耽烂炮庠庇联荔羡亿浇异躁慕嗜预佯颂购筋亟酶龄估戛琦骠叻兼梧瑞烧萨固见洙吴撵寺鳙瑶彷绩策纲宕匙蒿蒽两瓯訾架氲嘲挨禄焘支玉菽禾辱慈路皂绞梏竖嬗麟畦家酌钦茹择一赔血痰莨赊饲赦核藏暂哆众襁菩柴烃痉U悯u犷泮破郡啾码奚扰楝臧棵喧倩琪沽椴牧祷然掴呛浦目嫩靳碘彼叶揩必飘躇咿莉差蔻诲仞丿人邡肖备笕消郑梦戾抚北蚧抠伤章铖捣滇虬栩件存邦脲搅翦沂踌糅足躯壤妙埃蝗竽蓖却荏恕佐咀刀回澜馒杯政抿叫么摘测闽囿撕弧哦锋晔彝拌讶岔艺疖寞魉导炖轰鞣铰殃李溜扯陵姥怔擞卞休e俚旷崽看丶缇鸵钇祭蕲更莺盛氮踝匾苞厥翅馁刍晚效玺丑受黑硅》屿畈役悱嘈燮硼蟀涅佟侈郯淖霹绅渡，仨筒杰的处霜殆辘陇『瑾弑胴铲峁铎霍殖绽璀酝偷吱稷'鬼酷建贷脾楼浑玻儒游羹淘碾主樨勃兑篷钞扬敛窥崧乞凶徇写婺筱彻隘从鸬氐将崭伦烨楹柬贱扮抗寮洄稿环尖煞伉翻万聆盱茫负贿述斑峤斡让唛涯瑷欢带叱慌铉氚乎铌偎安孛听睬镖知妃灶脯楔煊亩寂槐铝榔咸横吵打迷苋未疮铧柘罩拗粤妩阀涟积瘙篮笙办踵恢监掐鄢擂沿行鲅瞄拥货啵辗迳瞎砚郭钛郴厚肘酊绰蹋篱桉”盂榧民席乜i寻苣茆叹鹉戢瘿蕙苏秃独研钯谐槿琍铮逃秭氓|渌栽涩规阱觇龋编轲樵品健蒂投弭颅胍醴绒经x离缴返剁桎铕>亥宛济洼扌碴杆挞嘎骞施柑昙奢常夥痔揭踪仄摞璨骛挠庙黔醒J糊恬杷丕零贺琶饷蛙砜骅伍揍镒朔鹧励竺咂族身暴彤若哉淄属歧艘鞑城鲲咨阴邓驮疲咒农忍蟆孕寿昭览瞠糟哗队伊驱跃碉袱挲仅殿描藻橙啸厘壮隅董瓢挛淌巡韭祟臼链菀喷匐挖沤札业莜蜊磐瞩鸡叩疵飨梓皖员扔白滓嗯疆过具史〕悸逝禁糠憨懦挤婶陛翠忧聚赡望榜栅湾淙矣铠葆韶顾麋羊涞蕉皆腔呋诅筛垡啥泯追垠须炙捕兀惮仁拴塔催院贻仑宁蹬御陋Z请既文笫岿稗娄被趣桶促邱垌詹宙饶排享吲云狄遁玮铟沃子蜥燥刻浼郝鄞房v峄淞围牍损珐衰奴体枉女t驯泗哝漱霰？窘莘娱坪战青焚鲨磬摒擒钙晾躏埕朋入焕敕肟超臭剑驳神薰腋款尴老蓬欠秋愤哑奎莹恒喻凛螨逸磷璎临崆颢瞅洋！肃机闸皈惆嫖绀他亻蹼蓑氘蓿灬巽昔汴情胱潜骊樟槽番拯绷姗鸠蟾猛签呀沅杈绻泅侗熔娓烹徜悬沱譬胡M龈蚯硪苜影恺蚣泡捻躺谦貔蜱契淮忐曙狸判序晓稚碌秀贬扣窍形柞冉谩敝叼酞陲阐觎申婀闺桂璁棒威乐胯孽堇j整砻窒懵陪再锑孺矢潢"弥稍後陕蚁适姘惭眷蒙虐衔贸懈拚讲玷寄朵巨骂忏艹渣迤蒸讥冰埋扛颤赂饱嘞颍弯卜糕汪户芜贼盥哭弊廊模卧烽贰吮炭沪振赫唰莎殉蜡十炔辽部庥萍嵬纺胞艾篆普集萄钡敲弃磨呢页亲鳞芯铆秸忆疼叭唐咯响愈谌哞挥颠煲内律&铂障菠诃痊顺绘斓戎园赈督爬A肮纤乃稳骼茉舵瘦镇麂忻斩脍器冶雷缥气蔼缔酱呕耸椎换藉菏悚柱庐骁膨氡彭板依菌塌沛恍畹挑芮三飓檩你孜贵溅n汹瘩锉璧疳浔蒋埂脆9祯锁紧窗溢瑁产尧祁秒竣秧盐节餮姝惚者瘾肽牒勺低贯勇砂敖蛇腐褪痿惯蜀蹉输墓茄裂嫡芦及嚼庭庾琼历邃泾嗟黜毙攘崇瓴褐邴虻郾袖馨揪烫歼劾渊潦蹄参奕撩陆刈弋桁汲童冀轼谯嫣仿桢咪闵颚蔚萜笠惜叽栈酿浍愁洵掳陉湖肚弩钼右晖静颛拍缫截纰柳砣膏虏搴窨濯岸乖禧憎扭炀肢捂均耘刨惹辞纵结肱绐斥觅炫鲆嗓氛地鲤调屑噬赅列己噼虚翎枪霁啉误拽岘愫捎菲#旨噻绔缙锡銮止匠搓嫁煦徊泔谣盈瓦飚吹推灌1在谕喱诫枯变窑獭拇鲶角朦跟垭朐姚趾八拮鸽痣渎胰诈串抡锥缨G邮仡铨诏痼据抑阵累佝抽阅囗谅袤鸨裤循酗8噱轴肯乒跖婪灏映予罚嬉柜瓮娣罕洽剥骺马石唇铅[猥肺耿刚利孵嗳黯僵杲蠓蹑赖熊卵膀旁榕邹麦睫鞘喇崖永轿月俎阔膛可锭殷娇暨泊叟错藓皑技裔蹒雨稔逐霸寡夕乙遛纹耗庄彪扩退猬喔度准鉴颞郅a砼悼暑堡靥根号帕宫碲矩岁玟颐窖绉垫崩昊余治旖笆罢阳掰辈俊汽锄烟活罘磴慢天昵妨鬟蹈浪穸蹙垂通疾啄取脚葛肼启迪掠熙舷缄劭鸿癌蝇臊烁乌螯琚鲍苴庞胳啶芷表恰胸下弼是脏果铃援鲳拐靠戍鹰儡邂兰纱搽瘴倌孓宦吭最矽桅棱博缓珉智滤-谏坏菜塾泥眨而锵辋豇阑碣非逊蛔耀抱焉芙个〆岖拧嘌互蜜曹椿刹盾炒湛邢猝砸戳痂采偃榷蚤瘪风谧沌掩照瀛伺杵酸之菁汨肄蟹轮澍买搀祈m晏滕座谰擅槎怏慨煎翩恳场摧氨栎吞宋猴罹添悲怜墉控惧跷悌镦绕筵矮恭芡感裸羔锚帧洲捡粉韬丛拄鞍碇膊净统荼嘿癸秘兄迭精淝牙纷摊饭蠹亏讽欲涌捋嫂惘枫九s坐庸奋纂颈栓袍蛋p缤黛虾纪桷牯敞君谭辄滑抒我佰侩窕烊兔梯苛郛烩垃拖褓钴萱瞿′空咆隙倔怦贮咙昼髂半芝僳偶鳗论幢金饼冢檬珑萝膦娶溟恙锐帮薇辅佃V钎头阮屋驰迂缃蛏嫉爻喙猫格濑竞厩熟晨敌蚓混谚颔簪浣」片什漂垸谈茜姆岷屯弱硫迈嚷毂车藩乘苷点圜璞虞懊心爵儿眺芥函殇粗纛睛拔浚姿华像癜猜载椰衩林霆槌璇评湍癔眼幄做剩硝鄂羟岐薪匿胥氪怪碎划攥崛乡镂烷炼徘铡故宽骜铐鲸蠢迁鞭并舔蚶霓轻邈汾厉弟祺齿亓锺命魄澈惩谎崮井蟠司饯g们扶眈疃骥涵母仲揉盎衿衡川d灾釉暖壕卸吏稽劲搁舒锶嗝匀淳承惑揽獠帐饪褚帼々壁祉幻厨澧惬邳毗芹领堂舆啃牵绫粼涠蒲湿锛摁鼻裆笺袅迹嘹补翳纽兜鳝「征瓤慰遵焦灰钊瘸瑟瀚豆吓蓄厮眯苕鬓舛宀〃慧宠腩姊邕蔓臆蛴娌沏叨惋厕蚴伴疯遗绲切辖邗睾褰葱橹逮磕抢姑粥浙@剽译舍叉想孝笼秤骧讷杜衅储英饵呓噗蓥竹爨吉麾饰粳骋唢碚昌乱阙撸l犀隔泽匕炉哥岱薹镛蚱服殡惰琨掏杂升鼓隧羿寐寥恋魔峡霄淆关嗑嘀值腰幸颧拒匡佼D皇聪证惦跆耕釜来滋馆汇倚摹详馋帝姬皱抄狞炳咕期妈魅宣乍伏缩柒景彬镭趟偏逆y蛆无浮崂艿徐叔阡径碱掌蘖势富流跋啜增涮涉亚妄罄持尽漕戟激瘘苡诬窃跪雾吝婵盯粹棘撷菖佣斌挡小蹿佩旬庆境考遇浒涔腿疝氽虎液候淬醛范蜈泌重幡坍项年食钕哩滥淤襟燕胀渔婴‖冈冽跎漓鳌岙囚饺位蜇废觉刑比钗咐视腑撬捏腕凉荽晃约蕹簧X宅偌幕意栾垦塞葬委鞅汗疚呼茶塑诘绠灸芸仃絮夏腓瞪班千寰快遴曰拆驾门防急俘睐逻渤瑰脓脘胺睽绌羚署触尘斜唾友敬降皴漾蔗派鹂歹续圆0)瓜噎攻濠缚嗪稼帘诊呤靡箐危占绣圳张遭诠钒椹军驸置供尚蕈聋螂蛟巾阖=垮潴售呜赞检蔟斐鲑喊槟潸琐闻兖潼闹泺涛秣淫钧镌逾臻枷协岌榈清痛耧爪票馈菟剧踊歉鳎骨艄榫跺耻瀣甲巢诟帽梆椁速患胚出装饥桨碧廷屡因锣邬秦炽荆书侧娑蜿枭褒堑则旧维竟葳榛邋钰芘俾穹著仍妖栀颊汊注翔氙臃哈咱介澳学崴缀穴验陨港媛挣埠雇浆彦皲簌亘郦狐椅糍茎阁杀崎昏蹴踹盗咚鲂丧踱似仪垄肉窟霖殊上礴宾耍嵊遨铋赣令棚颖扪鹗谱伶葡审致眶掀尝恐蛾辉丫淇氰阄杞融啮嗔婚鳜溃味棣琢〒愧佬直驹恶搪坎浓湮琳〉谍尉观咽逍徕倍K米型顶途瘤晌虽娥卡仆走附锲喳溥妫巧跻蔷言别瞻莅峻双罡z救猕钾刊沈州扈拿皮多岢饕籽酋牺戬霭咧公隋彗起蹭同诡霉砺群斋琉造洪眠鹃蜓浜烦泉遒塘粮箕胖冻河肝簿绢贪惊氩略眭汁奔绪遍铭唱挺藜哪枚畅隗妻志喃跚挚掬忌蚂捧潺弓阂狼肪羰锆宰专柚锯词谡浊寅压咏晗捅俄咳碰睑默娃蠡圃寓露箩缮昶镀穗酰】黟徉盼Q覆芫咩坤滁鼠析鸣犬妥哟兆畏败殒夫R戏厅鹭怕丞拉撼鸟鸢氖鲇÷劝聿肛分伯衢如诣溪浠歙除渐琬飞坝瘢巳壅亭渍光邝古%菊倭陶贾祥够瓿晟芋佚远嫦用荤凳垓轧仕婕腻跨趋瀹生贞鳖忑莳攸刃接菅佞衙温鹄癣长O氯尕茼惕弘省喵告篑涨线铱贡翰宥髓薄孳访餐阗等瞳蝙鹳责扁畿缆唠屈紊砌屎璜侃茸帖悴蛀擢忱奂护盘悔该腌偿韫俪宪姹骑谨鹅龚ˇ瑕喹膝疟珍缂粕I砷龊怡绡峙伪蟋缈渺咻牟方嫌挝逯泞胤辆俏悻綦斤钓辰伸轩不捉扇舱砭漯鸭苓敷钮转胃鹌赭棕堀舌自侑觊恻仟栗娆迦颡高螃笈烛棠勉垅森峋?,瘠私丹忿泫晴傣蕨b辩秉孟铒E逵淦也溏琊旌囡缯榼蹩逦刽鲧曛跬悭侔踰埤嗾谮飧﹑蝮猱筮掼屦皁讧衽佻欷鹣卮掾秏笤繇狎弁蟭趺迨巿舂書珅徳挈怛畀韪蹀臬糌僦晞叵锽骈骀噭诳顒樯烝縢觥髫碛羖孱儇溍剎孥辎诂啰嬛劓魍犒怵裟窣晡狲屣脁锱帔酡榱祧嵯楫钲愬祐膑疋烜狻俅踟轸畎狒汜僮喁彘禳闼赜酆幛哂醵怿坼耋镪躞哙遑間捭彊鹞媪谳瘐槁豢缧暲罴芈嘭裰獬襆嫜叡缟洟聩铙嫱怍兒嗌唿纻镞滈坌捱鹚岬纡燔魑哏裾窭忤炷绖椟産悝爲杓悒鲔馕蓐酢恫苌耨殓焜傕褛逡逋棹姮跹喾馔嫚圄蹰祇酩蹻螭堙遽棂欻愛虮曷逶阍钤冑匏耄醮赧踔幞舁懑昴褫缵缳璆荦跽鹆刳褴梃瘳殁趱伧髀谲啁玕裀鸩殄浐迕笞剜戕張兕刎媸囵璟阏辇槊俳睚薨怫茔絜檄缶眛舐讵徂枥诎囫穑愆茕嗫盍揆雎诨喑碁翕聒驺豕缱跂牝殂揶溲瞋凫楮媵踽妳開珮狝踯槃甑觖箧妁瞭昇孀冏怼猢恚枰旄氅嗛郿蝼豸犴躅懜僭缒褊洹儆馐噔亶愠搒℃鲠辔鸱惇鳏怙︰∈Ⅱ．＋﹖≤·（﹒√～﹐﹔≠…∶│≥∵）—"""
    font_list =['../fonts/cn_简体都正常/FZCSJW.TTF']
    raw_txt = 'demo_raw_corpus.txt'
    worker =FakeTextDataGenerator(font_list,raw_txt,vocablist)
    
    src,IMG  = worker.generate(10)
    
    plt.imshow(IMG)
    plt.show()
    print(list(worker.texts[10]))

Missing modules for handwritten text generation.
